# 1 简介
本篇教程主要是讲述如何使用Python操作Ne4j图数据库，实现py2neo、ipython-cypher、pandas、networkx、igraph以及jgraph包的一些基本操作

# 2 py2neo
`py2neo`是Neo4j的python驱动、安装：`pip install py2neo`
## 2.1 连接

In [1]:
# 这里刚开始自己多次尝试连接，都报错了，查了网上教程不通过，最后使用下面的格式通过了
from py2neo import Graph
graph=Graph(
    host="127.0.0.1",
    port=7687, # 可写可不写  bolt port
    user="neo4j",
    password="12345"
)

In [14]:
graph.delete_all()

## 2.2 节点
使用`Node`这个类来创建节点.第一个参数是`label`，然后后面的参数是节点的属性（`key-value`）

In [15]:
from py2neo import Node

In [16]:
qiang=Node("Person",name="闫强",age=24)
xiong=Node("Person",name="熊高雄",age=23)

ming=Node("Drink",name="小明",calories=90)
hong=Node("Drink",name="小红",calories=0)

graph.create(qiang|xiong|ming|hong)

## 2.3 关系
使用`Relationship`创建节点之间的关系

In [17]:
from py2neo import Relationship

In [20]:
graph.create(Relationship(qiang,"合作",ming))
graph.create(Relationship(xiong,"合作",hong))

## 2.4 Cypher
类似`SQL`语句，可以用来执行查询等各种操作

In [21]:
query="""
MATCH (person:Person)-[:合作]->(drink:Drink)
RETURN person.name AS name,drink.name AS drink
"""
data=graph.run(query)
for d in data:
    print(d)

<Record name='闫强' drink='小明'>
<Record name='熊高雄' drink='小红'>


## 2.5 参数化的Cypher
给Cypher语句传递`key-value`形式的参数

In [27]:
query = """
MATCH (p:Person)-[:合作]->(drink:Drink)
WHERE p.name = {name}
RETURN p.name AS name
"""

data = graph.run(query, name="闫强")

for d in data:
    print(d)

<Record name='闫强'>


# 3 jgraph
`jgraph`将元组数据描绘成3d图形

In [31]:
import jgraph

jgraph.draw([(1, 2), (2, 3), (3, 4), (4, 1), (4, 5), (5, 2)])

In [32]:
data = graph.run("MATCH (n)-->(m) RETURN ID(n), ID(m)")
data = [tuple(x) for x in data]

jgraph.draw(data)

# 4 igraph

In [42]:
from py2neo import Graph as PGraph
from igraph import Graph as IGraph 
from igraph import plot


query = """
MATCH (person:Person)-[:合作]->(drink:Drink)
RETURN person.name AS source, drink.name AS target
"""

data = graph.run(query)
tups = []

for d in data:
    tups.append((d["source"], d["target"]))

In [43]:
ig = IGraph.TupleList(tups)

print(ig)

IGRAPH UN-- 4 2 --
+ attr: name (v)
+ edges (vertex names):
闫强--小明, 熊高雄--小红


In [44]:
best = ig.vs.select(_degree = ig.maxdegree())["name"]
best

['闫强', '小明', '熊高雄', '小红']